In [13]:
import requests
import pandas as pd
import json
# https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart?id=1&range=7D&interval=hourly
url = 'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing'
# https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail?id=13411



params = {
    'start': 1,
    'limit': 100,
    'sortBy': 'market_cap',
    'sortType': 'desc',
    'convert': 'USD,BTC,ETH',
    'cryptoType': 'all',
    'tagType': 'all',
    'audited': 'false',
    'aux': 'ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap',
    # 'aux': 'ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,max_supply,circulating_supply,total_supply,volume_7d,volume_30d,self_reported_circulating_supply,self_reported_market_cap',
    'marketCapRange': '0~1000000'
}


headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'origin': 'https://coinmarketcap.com',
    'platform': 'web',
    'priority': 'u=1, i',
    'referer': 'https://coinmarketcap.com/',
    'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'x-request-id': '0752c2680fb44051b7372f063d95b111'
}




In [14]:
def get_data(url, params, headers):
    composed_url= f'{url}?{"&".join([f"{k}={v}" for k,v in params.items()])}'
    response = requests.get(url=composed_url, headers=headers)
    return response.json()

In [15]:
def get_pages(url, params, headers, pageLimit=1):
    data = get_data(url, params, headers)
    file_path = f"reports/coinmarketcap.csv"
    dataframe = pd.DataFrame(data['data']['cryptoCurrencyList'])
    dataframe.to_csv(file_path, mode='w', header=True)
    
    for i in range(2, pageLimit+1):
      try:
        params['start'] = i
        data = get_data(url, params, headers)
        dataframe = pd.DataFrame(data['data']['cryptoCurrencyList'])
        dataframe.to_csv(file_path, mode='a', header=False, index=False)
      except:
        continue
    return dataframe
   
data = get_pages(url, params, headers, pageLimit=50)

   
    

In [16]:
def gather_chart_data(id: int, range:str ='7D', interval: str='hourly', timeout: int=120):
    """
    Fetch chart data with a timeout
    
    Args:
        id: coin ID
        range: time range
        interval: data interval
        timeout: seconds to wait before timing out (default 2 minutes)
    """
    url = f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart'
    params = {
        'id': id,
        'range': range,
        'interval': interval
    }
    composed_url = f'{url}?{"&".join([f"{k}={v}" for k,v in params.items()])}'
    
    try:
        response = requests.get(url=composed_url, headers=headers, timeout=timeout)
        return response.json()
    except requests.exceptions.Timeout:
        print(f"Request timed out after {timeout} seconds for id {id}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for id {id}: {str(e)}")
        return None

In [17]:
import os
def get_charts(projects: pd.DataFrame, range: str = '7D', interval: str = 'hourly'):
    for _, project in projects.iterrows():
        try:
            raw_id = project['id']
            raw_slug = project['slug']
            
            # Check if id is numeric - if not, swap with slug
            if not str(raw_id).isnumeric() and str(raw_slug).isnumeric():
                project_id = raw_slug
                project_slug = raw_id
            else:
                project_id = raw_id
                project_slug = raw_slug
                
            print(f"Getting chart data for {project_slug} ({project_id})")
            
            data = gather_chart_data(project_id, range, interval)
            dataframe = pd.DataFrame(data['data']['points'])
            dataframe['id'] = project_id
            dataframe['slug'] = project_slug
            
            os.makedirs(f'reports/{project_slug}', exist_ok=True)
            file_path = f'reports/{project_slug}/chart_data.json'
            with open(file_path, 'w') as f:
                json.dump(data, f, indent=2)    
        
        except Exception as e:
            print(f"Failed to get chart data for {project_slug}: {e}")
            continue

# Read the CSV file and get the projects
df = pd.read_csv("reports/coinmarketcap.csv")
projects = df[['id', 'slug']].iloc[:250]


# Call the get_charts function
get_charts(projects, range='7D', interval='hourly')


Getting chart data for veloce-vext (27862)
Getting chart data for weebs (32816)
Getting chart data for nabox (9653)
Getting chart data for auto (8387)
Getting chart data for milady-wif-hat (30071)
Getting chart data for game-of-memes (31261)
Getting chart data for rebel-bots (18354)
Getting chart data for ally (5011)
Getting chart data for fidex-token (3800)
Getting chart data for mindai (30053)
Getting chart data for ethernity-cloud (11815)
Getting chart data for calamari-network (15305)
Getting chart data for contracoin (5313)
Getting chart data for moss-carbon-credit (8826)
Getting chart data for pussy-financial (9639)
Getting chart data for xels (16962)
Getting chart data for bolt (3843)
Getting chart data for sc-corinthians-fan-token (11446)
Getting chart data for velaspad (11654)
Getting chart data for thorstarter (10260)
Getting chart data for crowns (8365)
Getting chart data for meta-apes-peel (21013)
Getting chart data for etho-protocol (3452)
Getting chart data for edu3labs (

In [18]:
def get_details(id: int):
    url = f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail'
    params = {
        'id': id
    }
    composed_url= f'{url}?{"&".join([f"{k}={v}" for k,v in params.items()])}'
    response = requests.get(url=composed_url, headers=headers)
    return response.json()

In [19]:
import json
import os

def get_project_details(projects: pd.DataFrame):
    for _, project in projects.iterrows():
        try:
            raw_id = project['id']
            raw_slug = project['slug']
            if not str(raw_id).isnumeric() and str(raw_slug).isnumeric():
                project_id = raw_slug
                project_slug = raw_id
            else:
                project_id = raw_id
                project_slug = raw_slug
            
            data = get_details(project_id)
            
            # Create directory if it doesn't exist
            os.makedirs(f'reports/{project_slug}', exist_ok=True)
            
            # Save raw JSON data
            file_path = f'reports/{project_slug}/details.json'
            with open(file_path, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"Details saved to {file_path}")
            
        except Exception as e:
            print(f"Failed to get details for {project_slug}")
            print(f"Error: {str(e)}")
            continue

df = pd.read_csv("reports/coinmarketcap.csv")
projects = df[['id', 'slug']].iloc[:250]
get_project_details(projects)

Details saved to reports/veloce-vext/details.json
Details saved to reports/weebs/details.json
Details saved to reports/nabox/details.json
Details saved to reports/auto/details.json
Details saved to reports/milady-wif-hat/details.json
Details saved to reports/game-of-memes/details.json
Details saved to reports/rebel-bots/details.json
Details saved to reports/ally/details.json
Details saved to reports/fidex-token/details.json
Details saved to reports/mindai/details.json
Details saved to reports/ethernity-cloud/details.json
Details saved to reports/calamari-network/details.json
Details saved to reports/contracoin/details.json
Details saved to reports/moss-carbon-credit/details.json
Details saved to reports/pussy-financial/details.json
Details saved to reports/xels/details.json
Details saved to reports/bolt/details.json
Details saved to reports/sc-corinthians-fan-token/details.json
Details saved to reports/velaspad/details.json
Details saved to reports/thorstarter/details.json
Details save